In [5]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel_transfer, find_state, directory_scratch, directory_project, directory_archive
import dropbox
from amftrack.transfer.functions.transfer import upload
from subprocess import call
from tqdm import tqdm
from time import sleep

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
API = str(np.load(os.getenv('HOME')+'/pycode/API_drop.npy'))


In [13]:
dbx = dropbox.Dropbox(API)
response = dbx.files_list_folder("",recursive = True)
for file in response.entries:
    print (file.name.split(".")[0])

prince_data
94_20201123
20201124_0000_Plate39
20201210_1156_Plate39
20201124_0301_Plate39
20201123_2300_Plate39
20201124_1001_Plate39
20201124_0201_Plate39
20201124_1101_Plate39
20201123_2100_Plate39
20201123_1253_Plate39
20201207_0924_Plate39
20201123_1900_Plate39
20201125_1902_Plate39
20201123_2000_Plate39
20201125_1502_Plate39
20201123_1701_Plate39
20201207_0124_Plate39
20201123_1400_Plate39
20201124_0901_Plate39
20201207_0523_Plate39
20201125_1602_Plate39
20201123_1601_Plate39
20201125_0502_Plate39
20201123_1801_Plate39
20201124_0801_Plate39
20201123_1501_Plate39
20201125_1802_Plate39
20201124_1302_Plate39
20201125_0802_Plate39
20201126_0602_Plate39
20201124_2202_Plate39
20201124_1402_Plate39
20201124_2002_Plate39
20201123_2200_Plate39
20201125_0702_Plate39
20201126_0802_Plate39
20201124_0601_Plate39
20201124_0401_Plate39
20201126_1002_Plate39
20201124_1702_Plate39
20201124_2102_Plate39
20201126_0702_Plate39
20201125_1102_Plate39
20201124_1602_Plate39
20201124_0501_Plate39
20201124

In [14]:
list_dir = [file.name.split(".")[0] for file in response.entries]

In [15]:
list_dir

['prince_data',
 '94_20201123',
 '20201124_0000_Plate39',
 '20201210_1156_Plate39',
 '20201124_0301_Plate39',
 '20201123_2300_Plate39',
 '20201124_1001_Plate39',
 '20201124_0201_Plate39',
 '20201124_1101_Plate39',
 '20201123_2100_Plate39',
 '20201123_1253_Plate39',
 '20201207_0924_Plate39',
 '20201123_1900_Plate39',
 '20201125_1902_Plate39',
 '20201123_2000_Plate39',
 '20201125_1502_Plate39',
 '20201123_1701_Plate39',
 '20201207_0124_Plate39',
 '20201123_1400_Plate39',
 '20201124_0901_Plate39',
 '20201207_0523_Plate39',
 '20201125_1602_Plate39',
 '20201123_1601_Plate39',
 '20201125_0502_Plate39',
 '20201123_1801_Plate39',
 '20201124_0801_Plate39',
 '20201123_1501_Plate39',
 '20201125_1802_Plate39',
 '20201124_1302_Plate39',
 '20201125_0802_Plate39',
 '20201126_0602_Plate39',
 '20201124_2202_Plate39',
 '20201124_1402_Plate39',
 '20201124_2002_Plate39',
 '20201123_2200_Plate39',
 '20201125_0702_Plate39',
 '20201126_0802_Plate39',
 '20201124_0601_Plate39',
 '20201124_0401_Plate39',
 '2020

In [16]:
def get_current_folders(directory,dropbox = False):
    plate_info = get_data_info()
    if dropbox:
        dbx = dropbox.Dropbox(API)
        response = dbx.files_list_folder("",recursive = True)
        list_dir = [file.name.split(".")[0] for file in response.entries]
    else:
        listdir = os.listdir(directory)
    return(plate_info.loc[np.isin(plate_info['folder'],listdir)&(plate_info['total_path']==directory)])